In [31]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [12]:
from dk_weekly_update import *
from dk_weekly_update import DkWeeklyUpdate as dk

In [124]:
qb = pd.read_csv('input/qb/qb_loc_cleaned.csv')
qb.sort_values(['Name','games'], inplace=True)
qb.tail()

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
2349,37,2018,Tyrod Taylor,CLE,3,NYJ,14,28.6,19,0,22,1.0,1.0,2.96
2350,1,2014,Zach Mettenberger,TEN,8,HOU,41,65.9,299,2,12,1.0,1.0,17.86
2351,2,2014,Zach Mettenberger,TEN,11,PIT,24,62.5,263,2,9,1.0,1.0,17.52
2352,3,2014,Zach Mettenberger,TEN,12,PHI,39,51.3,345,2,30,1.0,1.0,23.60
2353,4,2015,Zach Mettenberger,TEN,15,NE,28,71.4,242,2,15,0.0,0.0,16.18


In [131]:
wk7 = pd.read_csv('input/qb/qb_wk7.csv')
wk7.columns

Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGameDraftKings', 'FantasyPointsDraftKings'],
      dtype='object')

In [178]:
wk8 = pd.read_csv('input/qb/qb_wk8.csv')

In [179]:
qb[qb['Name'] =='AJ McCarron']

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
0,1,2015,AJ McCarron,CIN,14,PIT,32,68.8,280,2,24,1.0,0.0,18.0


In [193]:
location = pd.read_csv('input/location.csv')
location.tail()

,Unnamed: 0,date,day,week,home,away,year
1653,116,"October 28, 2018",Sun,8,OAK,IND,2018
1654,117,"October 28, 2018",Sun,8,LAR,GB,2018
1655,118,"October 28, 2018",Sun,8,ARI,SF,2018
1656,119,"October 28, 2018",Sun,8,MIN,NO,2018
1657,120,"October 29, 2018",Mon,8,BUF,NE,2018


In [166]:
def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

In [126]:
wk7_def = pd.read_csv('input/def/wk7_def.csv')

In [180]:
position_dict = {'qb':'DraftKingsQuarterbackFantasyPointsAllowedAverage',
                    'wr':'DraftKingsWideReceiverFantasyPointsAllowedAverage',
                    'rb':'DraftKingsRunningbackFantasyPointsAllowedAverage',
                    'te':'DraftKingsTightEndFantasyPointsAllowedAverage'}

team_dict = {'New England Patriots':'NE',
            'Green Bay Packers':'GB',
            'Tampa Bay Buccaneers':'TB',
            'Kansas City Chiefs':'KC',
            'San Diego Chargers':'SD',
            'St. Louis Rams':'STL',
            'New Orleans Saints':'NO',
            'New York Jets':'NYJ',
            'New York Giants':'NYG',
            'San Francisco 49ers':'SF',
            'Los Angeles Rams':'LAR',
            'Jacksonville Jaguars':'JAX',
            'Los Angeles Chargers':'LAC'}

In [177]:
qb.head()

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
0,1,2015,AJ McCarron,CIN,14,PIT,32,68.8,280,2,24,1.0,0.0,18.00
1,1,2012,Aaron Rodgers,GB,1,SF,44,68.0,303,2,5,1.0,1.0,22.82
2,2,2012,Aaron Rodgers,GB,2,CHI,32,69.0,219,1,31,1.0,1.0,10.16
3,3,2012,Aaron Rodgers,GB,3,SEA,39,67.0,223,0,1,0.0,0.0,10.62
4,4,2012,Aaron Rodgers,GB,4,NO,41,76.0,319,4,29,1.0,1.0,28.06


In [181]:
wk8.rename(columns ={
                      'PassingAttempts':'att',
                      'PassingCompletionPercentage':'comp%',
                      'PassingYards':'yds',
                      'PassingTouchdowns':'td',
                      'FantasyPointsDraftKings':'points'},inplace=True)
wk8.columns


Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent',
       'PassingCompletions', 'att', 'comp%', 'yds', 'PassingYardsPerAttempt',
       'td', 'int', 'PassingRating', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGameDraftKings', 'points'],
      dtype='object')

In [182]:
for col in wk8.columns:
    if 'Rushing' in col:
        wk8.drop(col,axis=1,inplace=True)
wk8.columns

Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent',
       'PassingCompletions', 'att', 'comp%', 'yds', 'PassingYardsPerAttempt',
       'td', 'int', 'PassingRating', 'FantasyPointsPerGameDraftKings',
       'points'],
      dtype='object')

In [185]:
wk8.drop(['PassingInterceptions','PassingYardsPerAttempt','PassingCompletions', 'FantasyPointsPerGameDraftKings', 'Rank','Position', 'PassingRating'],axis=1, inplace=True)
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td',
       'points'],
      dtype='object')

In [186]:
wk8

,Name,Team,Week,Opponent,att,comp%,yds,td,points
0,Deshaun Watson,HOU,8,MIA,20,80.0,239,5,30.96
1,Patrick Mahomes,KC,8,DEN,34,70.6,303,4,30.02
2,Cam Newton,CAR,8,BAL,29,72.4,219,2,27.96
3,Derek Carr,OAK,8,IND,28,75.0,244,3,27.76
4,Jared Goff,LAR,8,GB,35,54.3,295,3,26.50
5,Carson Wentz,PHI,8,JAX,30,70.0,286,3,25.64
6,Kirk Cousins,MIN,8,NO,41,75.6,359,2,24.66
7,Russell Wilson,SEA,8,DET,17,82.4,248,3,23.42
8,Andrew Luck,IND,8,OAK,31,71.0,239,3,23.36
9,Matthew Stafford,DET,8,SEA,40,67.5,310,2,22.00


In [27]:
def_2018.head()

,Rank,Name,Team,Position,Week,Opponent,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Chicago Bears,CHI,DST,8,NYJ,12.32,10.0,20.2,8.6,5,56.12
1,2,Arizona Cardinals,ARI,DST,8,SF,12.50,9.8,28.3,10.7,7,68.30
2,3,New England Patriots,NE,DST,8,BUF,10.42,22.0,29.0,9.0,9,79.42
3,4,Pittsburgh Steelers,PIT,DST,8,CLE,15.10,16.5,29.6,12.4,7,80.60
4,5,Minnesota Vikings,MIN,DST,8,NO,10.86,38.8,17.9,0.0,16,83.56


In [80]:
wk7.head()

,Name,Team,Week,Opponent,att,comp%,yds,yds/att,td,int,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,points
0,Patrick Mahomes,KC,7,CIN,39,71.8,358,9.2,4,1,4,45,11.2,0,36.82
1,Mitch Trubisky,CHI,7,NE,50,52.0,333,6.7,2,2,6,81,13.5,1,36.42
2,Jameis Winston,TB,7,CLE,52,61.5,365,7.0,0,2,10,55,5.5,1,26.10
3,Cam Newton,CAR,7,PHI,39,64.1,269,6.9,2,0,7,49,7.0,0,25.66
4,Eli Manning,NYG,7,ATL,38,71.1,399,10.5,1,0,3,7,2.3,0,23.66


In [83]:
for idx, team in enumerate(def_2018['Team']):
    print(team)

HOU
NE
MIN
CHI
NYG
ARI
PIT
LAR
BUF
KC
WAS
CLE
CAR
PHI
TB
SF
NYJ
SEA
OAK
DET
NO
JAX
GB
IND
BAL
DEN
MIA
CIN


In [187]:
wk8['year'] =2018
def_2018.sort_values(position_dict['qb'], inplace=True)
def_2018.reset_index(inplace=True, drop=True)
def_2018['Rank'] = def_2018.index + 1
def_2018.index = def_2018['Team']

# Map defensive ranking to each player in stats df
wk8['def_rk'] = list(map(lambda x: def_2018.loc[x]['Rank'], wk8['Opponent']))

wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td',
       'points', 'year', 'def_rk'],
      dtype='object')

In [74]:
qb[qb['year']==2018]['Week'].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [168]:
wk8.head()

,Name,Team,Week,Opponent,att,comp%,yds,yds/att,td,points,year,def_rk
0,Deshaun Watson,HOU,8,MIA,20,80.0,239,12.0,5,30.96,2018,27
1,Patrick Mahomes,KC,8,DEN,34,70.6,303,8.9,4,30.02,2018,26
2,Cam Newton,CAR,8,BAL,29,72.4,219,7.6,2,27.96,2018,25
3,Derek Carr,OAK,8,IND,28,75.0,244,8.7,3,27.76,2018,24
4,Jared Goff,LAR,8,GB,35,54.3,295,8.4,3,26.50,2018,23


In [69]:
wk8.shape

(74, 12)

In [194]:
loc = location[location['year']==2018]
loc = loc[loc['week']==8]
loc.shape

(14, 7)

## Home or Away

In [195]:
home_lst = []
for team in wk8['Team']:
    if team in list(loc['home']):
        home_lst.append('home')           
    else:
        home_lst.append('away')
wk8['h/a'] = home_lst

In [196]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td',
       'points', 'year', 'def_rk', 'h/a'],
      dtype='object')

## Surface

In [198]:
surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [199]:
wk8.index = wk8['Team']

# Add the type of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(wk8['Team']):
    if wk8.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[wk8.iloc[idx]['Opponent']])


wk8['surface'] = surface

## Encode

In [200]:
wk8 = one_hot(wk8)

In [201]:
wk8.drop(['h/a_0','surface_1','surface','h/a','surface_encoded','h/a_encoded'], axis=1, inplace=True)
wk8.shape

(38, 13)

In [203]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td',
       'points', 'year', 'def_rk', 'h/a_1', 'surface_0'],
      dtype='object')

In [109]:
qb.shape

(2354, 14)

In [47]:
qb.head()

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
0,1,2015,AJ McCarron,CIN,14,PIT,32,68.8,280,2,24,1.0,0.0,18.00
1,1,2012,Aaron Rodgers,GB,1,SF,44,68.0,303,2,5,1.0,1.0,22.82
2,2,2012,Aaron Rodgers,GB,2,CHI,32,69.0,219,1,31,1.0,1.0,10.16
3,3,2012,Aaron Rodgers,GB,3,SEA,39,67.0,223,0,1,0.0,0.0,10.62
4,4,2012,Aaron Rodgers,GB,4,NO,41,76.0,319,4,29,1.0,1.0,28.06


In [53]:
qb[qb['Name']=='Aaron Rodgers']

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
0,1,2012,Aaron Rodgers,GB,1,SF,44,68.0,303,2,5,1.0,1.0,22.82
39,2,2012,Aaron Rodgers,GB,2,CHI,32,69.0,219,1,31,1.0,1.0,10.16
74,3,2012,Aaron Rodgers,GB,3,SEA,39,67.0,223,0,1,0.0,0.0,10.62
113,4,2012,Aaron Rodgers,GB,4,NO,41,76.0,319,4,29,1.0,1.0,28.06
151,5,2012,Aaron Rodgers,GB,5,IND,32,63.0,235,3,13,0.0,0.0,25.10
188,6,2012,Aaron Rodgers,GB,6,HOU,37,65.0,338,6,10,0.0,0.0,39.22
221,7,2012,Aaron Rodgers,GB,7,LAR,37,81.0,342,3,11,0.0,1.0,25.68
252,8,2012,Aaron Rodgers,GB,8,JAX,35,63.0,186,2,14,1.0,1.0,13.84
287,9,2012,Aaron Rodgers,GB,9,ARI,30,47.0,218,4,23,1.0,1.0,26.02
357,10,2012,Aaron Rodgers,GB,11,DET,27,70.0,236,2,7,0.0,0.0,15.74


In [175]:
for name in qb['Name'].unique():
    print(qb[qb['Name']==name]['games'].max())

73
65
67
70
67
23
18
2
74
57
18
20
42
80
62
17
46
58
7
16
6
22
14
10
79
23
75
14
46
77
28
76
48
56
37
12
78
40
37
17
2
4
2
4
4
18
4
9
10
9
7
45
1
4
10
6
2
2
29
7
2
5
2
5
1
21
1
2
1
11
16
1
12
23
27
1
3
38
2
41
5
13
2
4
2
33
32
2
1
1
25
9
29
9
1
2
17
3
1
9
12
6
7
3
10
2
1
1
6
6
2
6
6
5
4
5
1
1
1


## Games

In [204]:
games_lst = []
temp = wk8.set_index('Name')
for name in temp.index:
    if name in list(qb['Name'].unique()):
        games_lst.append(qb[qb['Name']==name]['games'].max()+1)
    else:
        games_lst.append(1)
    

In [205]:
wk8['games'] = games_lst

In [206]:
wk8

,Name,Team,Week,Opponent,att,comp%,yds,td,points,year,def_rk,h/a_1,surface_0,games
0,Deshaun Watson,HOU,8,MIA,20,80.0,239,5,30.96,2018,27,1.0,0.0,14
1,Patrick Mahomes,KC,8,DEN,34,70.6,303,4,30.02,2018,26,1.0,1.0,8
2,Cam Newton,CAR,8,BAL,29,72.4,219,2,27.96,2018,25,1.0,1.0,76
3,Derek Carr,OAK,8,IND,28,75.0,244,3,27.76,2018,24,1.0,1.0,39
4,Jared Goff,LAR,8,GB,35,54.3,295,3,26.50,2018,23,1.0,1.0,19
5,Carson Wentz,PHI,8,JAX,30,70.0,286,3,25.64,2018,22,0.0,1.0,27
6,Kirk Cousins,MIN,8,NO,41,75.6,359,2,24.66,2018,21,1.0,0.0,47
7,Russell Wilson,SEA,8,DET,17,82.4,248,3,23.42,2018,20,0.0,0.0,77
8,Andrew Luck,IND,8,OAK,31,71.0,239,3,23.36,2018,19,0.0,1.0,69
9,Matthew Stafford,DET,8,SEA,40,67.5,310,2,22.00,2018,18,1.0,0.0,77


In [116]:
wk7['year']=2018

## Reshape

In [211]:
cols = wk8.columns.tolist()
cols = cols[-1:] +cols[-5:-4] +cols[:8] + cols[-4:-1] + cols[-6:-5]
wk8 = wk8[cols]
wk8

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
0,14,2018,Deshaun Watson,HOU,8,MIA,20,80.0,239,5,27,1.0,0.0,30.96
1,8,2018,Patrick Mahomes,KC,8,DEN,34,70.6,303,4,26,1.0,1.0,30.02
2,76,2018,Cam Newton,CAR,8,BAL,29,72.4,219,2,25,1.0,1.0,27.96
3,39,2018,Derek Carr,OAK,8,IND,28,75.0,244,3,24,1.0,1.0,27.76
4,19,2018,Jared Goff,LAR,8,GB,35,54.3,295,3,23,1.0,1.0,26.50
5,27,2018,Carson Wentz,PHI,8,JAX,30,70.0,286,3,22,0.0,1.0,25.64
6,47,2018,Kirk Cousins,MIN,8,NO,41,75.6,359,2,21,1.0,0.0,24.66
7,77,2018,Russell Wilson,SEA,8,DET,17,82.4,248,3,20,0.0,0.0,23.42
8,69,2018,Andrew Luck,IND,8,OAK,31,71.0,239,3,19,0.0,1.0,23.36
9,77,2018,Matthew Stafford,DET,8,SEA,40,67.5,310,2,18,1.0,0.0,22.00


## Concatenate

In [212]:
qb = pd.concat([qb,wk8])
qb.tail()

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,def_rk,h/a_1,surface_0,points
33,9,2018,Joe Webb,HOU,8,MIA,0,0.0,0,0,27,1.0,0.0,0.0
34,14,2018,Teddy Bridgewater,NO,8,MIN,0,0.0,0,0,3,0.0,0.0,0.0
35,19,2018,Brandon Weeden,HOU,8,MIA,0,0.0,0,0,27,1.0,0.0,-0.1
36,2,2018,Joshua Dobbs,PIT,8,CLE,0,0.0,0,0,12,1.0,1.0,-0.1
37,22,2018,Brian Hoyer,NE,8,BUF,0,0.0,0,0,9,0.0,0.0,-0.2


In [213]:
qb.to_csv('input/qb/qb_loc_cleaned.csv', index=False)

In [5]:
# Parse upcoming week file by position

def clean_current(csv):# Read in files

    df_upcoming = pd.read_csv(csv)
    def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

    # Obtain home and away attributes
    df_upcoming['year']=2018
    df_upcoming['home'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[1]).split()[0])
    df_upcoming['away'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[0]))

    rb = []
    wr = []
    te = []
    qb= []
    for idx,pos in enumerate(df_upcoming['Position']):
        if pos == 'RB':
            rb.append(idx)
        elif pos =='WR':
            wr.append(idx)
        elif pos =='TE':
            te.append(idx)
        elif pos =='QB':
            qb.append(idx)


    # Sort current week dataframe

    qb_df = df_upcoming.iloc[qb]
    rb_df = df_upcoming.iloc[rb]
    wr_df = df_upcoming.iloc[wr]
    te_df = df_upcoming.iloc[te]
    
    return qb_df,wr_df,te_df, rb_df

In [26]:
# Prepare defense dataframe by sorting by relevant attribute reseting the indexand using rank as the reset dataframe index +1

def add_defensive_rank(df, df1, position):
    df1.sort_values(position_dict[position], inplace=True)
    df1.reset_index(inplace=True, drop=True)
    df1['Rank']= df1.index+1
    df1.index=df1['Team']

# Map defensive rnking to eack player in the statist  ical dataframe 

    df['def_rk'] = list(map(lambda x: df1.loc[x]['Rank'], df['Opponent']))
    
    return df


In [23]:
# A function designed to add the Draftkings fantasy football salary to every players statistics dataframe

def add_salary(df,df2):
    df = df.sort_values(['Week','PlayerID'])
    df = df.drop(['FantasyPointsPerGameDraftKings','Rank','Position'], axis=1)
    df.reset_index(inplace=True, drop=True)
    df2 = df2.sort_values(['Week','PlayerID'])
    df2 = df2.drop(['Rank','Position'], axis=1)
    df2.reset_index(inplace=True, drop=True)
    df['salary'] = df2['DraftKingsSalary']
    
    return df

    

In [41]:
# One hot encode all non numeric attributes


def one_hot(df):
    df.reset_index(inplace=True, drop=True)
    encoder = LabelEncoder()
    df['h/a_encoded'] = encoder.fit_transform(df['h/a'])
    df['surface_encoded'] = encoder.fit_transform(df['surface'])


    home =OneHotEncoder()
    surface = OneHotEncoder()

    X = home.fit_transform(df['h/a_encoded'].values.reshape(-1,1)).toarray()
    Xs = surface.fit_transform(df['surface_encoded'].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["h/a_"+str(int(i)) for i in range(X.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)


    dfOneHot = pd.DataFrame(Xs, columns = ['surface_'+str(int(i)) for i in range(Xs.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)

    return df

In [40]:
csv = 'input/qb/DKSalaries.csv'
qb7, wr7, te7, rb7 = clean_current(csv)
